
<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=10>
    Deep Learning - HW4 <br>
<font color=2565AE size=5>
    Electrical Engineering Department <br>
    winter 2024<br>
<font color=3C99D size=5>
    Practical Assignment 3 <br>
<font color=696880 size=4>
    Amirabbas Afzali 

____

# Personal Data

In [2]:
# Set your student number
student_number = '401110437'
Name = 'Parsa'
Last_Name = 'Ghezelbash'

# Rules
- Make sure that all of your cells can be run perfectly. 
- Try to minimize your use of ChatGPT (or any other AI assistant) as much as possible.
- You must create a report for this task in PDF format and explain the main results.

---

## **Introduction**

Large Language Models (LLMs) are a class of deep learning models designed for processing and generating natural language. These models are trained using large amounts of textual data and utilize architectures based on transformers. Some of the applications of these models include text generation, machine translation, text summarization, question answering, and text classification.

### *Encoder-Decoder LLMs*

One of the common architectures in large language models is the Encoder-Decoder architecture. In this architecture, the encoder processes an input sequence and maps it to a latent space. Then, the decoder uses this latent space to generate an output sequence. Models like T5 [1] (Text-to-Text Transfer Transformer) use this architecture to perform various tasks. In T5, all tasks are expressed in a "text-to-text" format, meaning both input and output are text. This model has capabilities such as translation, summarization, and text classification. One of the advantages of the Encoder-Decoder architecture is that it allows the encoder to utilize information from both before and after a word to gain a more comprehensive understanding of the text.

### *Decoder-only LLMs*

Decoder-only models, such as GPT-2, GPT-3, and LLaMA [2], unlike the Encoder-Decoder architecture, only use the decoder part. These models use an autoregressive mode, meaning they predict the next token based on previous tokens. These models are highly efficient for text generation and have found widespread applications today.

Advantages of Decoder-only Models

- Efficiency: Decoder-only models are more efficient than Encoder-Decoder models due to the absence of an independent encoder. This makes them require fewer computational and memory resources.
- Simplicity: Due to their autoregressive nature, these models can easily generate sequences in order.
- Scalability: Due to their simpler architecture, these models can be scaled to much larger sizes.


However, one of the drawbacks of these models is that they can only utilize information from tokens before the current token and cannot use tokens that come after for prediction. This limitation is significant in tasks like classification or translation, where a full understanding of the sequence is needed.



## **Objective of the Exercise**


In this exercise, the goal is to convert a generative Decoder-only language model into an encoder and evaluate its performance on a binary sentiment classification task. The main aim is to modify the Decoder-only model so that it can function as an encoder and better handle tasks requiring bidirectional understanding.

## **In this exercise, you should:**


### In this exercise, you should:

1. **Import a Decoder-only model** and load the weights of a pre-trained version of the model.
2. **Generate several outputs from the model**, and include at most 10 sample outputs in your report for different inputs.  
   You should also briefly explain the effects of key configurations in text generation, including:  
   - `Temperature`
   - `top_k`
   - `top_p`
   - `repetition_penalty`
   - `num_beams`
   - `no_repeat_ngram_size`
3. **Load the SST-2 dataset**, which is part of the GLUE benchmark for sentiment classification.  
   - Note that the model’s output depends on the number of input tokens. 
   - Apply necessary padding to the dataset after loading it to allow for parallel execution of the model.
4. **Remove the model’s final layer**, which outputs to the size of the model’s dictionary.  
   - Use the embedding vector of the first token (CLS token) for classification.
5. As observed in the previous step, sometimes the embedding vector of the first token does not provide a good representation of the entire input text.  
   - **Add a linear layer** with the same input and output dimensions on top of the encoder's output, and use the output of this linear layer (corresponding to the CLS token) for classification.  
   - This step aggregates information of different tokens to get a comprehensive understanding of the input text.
6. **Instead of the linear layer** in the previous section, use a **bidirectional attention layer** with a custom number of heads (preferably 12).
7. **Repeat step 6** using **left-to-right unidirectional attention** and **right-to-left unidirectional attention**.
8. **Load a pre-trained decoder** (preferably BERT-base) and report its **zero-shot performance** (i.e., without needing to train the model) on the test data.

## **Evaluation:**

In this exercise, for each of sections 4, 5, 6, 7, and 8, you need to plot the confusion matrix corresponding to the model's performance on the test data. Additionally, you should plot two separate graphs showing the training loss and the accuracy of the trained models, and compare them with each other, providing an appropriate analysis of your results. Also, note that high accuracy is not expected for sections 4 and 5, but the correctness of your code will be checked. However, for sections 6 and 7, higher accuracy (around 90%) is expected.













## **Let's go:**


Load `gpt2` model:

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Model
import torch
from torch import nn


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to("cuda")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Load `sst-2` dataset:

In [4]:
from datasets import load_dataset

# Load the SST-2 dataset from Hugging Face 
dataset = load_dataset("glue", "sst2")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

go ahead:

In [9]:
# Example inputs for text generation
prompts = [
    "Artificial intelligence is",
    "The future of AI will",
    "Deep learning models are",
]

# Generate text
outputs = []
for prompt in prompts:
    tokenizer.pad_token = tokenizer.eos_token  # Set pad token as eos token
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    attention_mask = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).attention_mask
    generated_ids = model.generate(
        input_ids=input_ids.to("cuda"),
        attention_mask=attention_mask.to("cuda"),
        max_length=50,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        temperature=0.7,  # Example temperature
        top_k=50,         # Example top_k
        top_p=0.9,        # Example top_p
        repetition_penalty=1.2,
        num_beams=5,      # Example num_beams
        no_repeat_ngram_size=2
    )
    outputs.append(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

# Print outputs
for i, output in enumerate(outputs):
    print(f"Prompt: {prompts[i]}")
    print(f"Generated Text: {output}\n")


Prompt: Artificial intelligence is
Generated Text: Artificial intelligence is one of the most important advances in the field of artificial intelligence, and it has been a major contributor to the advancement of our understanding of life on Earth.

In this article, we are going to look at a number of

Prompt: The future of AI will
Generated Text: The future of AI will be determined by what happens in the next few years," he said.

Explore further: Artificial intelligence could change the way we live, work and play

Prompt: Deep learning models are
Generated Text: Deep learning models are often used to predict the future.

In this article, we will explore how to use these models to build predictive models that can be used in real-world situations. We will also look at how we can use them to



In [10]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

# Load SST-2 dataset
dataset = load_dataset("glue", "sst2")

# Tokenize and pad
def tokenize(batch):
    return tokenizer(batch["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Prepare DataLoaders
train_loader = DataLoader(tokenized_dataset["train"], batch_size=32, collate_fn=data_collator)
validation_loader = DataLoader(tokenized_dataset["validation"], batch_size=32, collate_fn=data_collator)
test_loader = DataLoader(tokenized_dataset["test"], batch_size=32, collate_fn=data_collator)


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [12]:
from transformers import GPT2Model

# Load GPT-2 without the language modeling head
base_model = GPT2Model.from_pretrained("gpt2").to("cuda")

# Extract CLS token embedding
def extract_cls_embedding(batch):
    inputs = batch["input_ids"].to("cuda")
    attention_mask = batch["attention_mask"].to("cuda")
    outputs = base_model(input_ids=inputs, attention_mask=attention_mask)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token embedding
    return cls_embedding



In [13]:
class GPT2Classifier(nn.Module):
    def __init__(self, hidden_dim, num_labels):
        super().__init__()
        self.linear = nn.Linear(hidden_dim, num_labels)
    
    def forward(self, cls_embedding):
        return self.linear(cls_embedding)

# Example: Add linear layer
classifier = GPT2Classifier(hidden_dim=base_model.config.hidden_size, num_labels=2).to("cuda")


In [ ]:
class BiAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)

    def forward(self, embeddings):
        attn_output, _ = self.attention(embeddings, embeddings, embeddings)
        return attn_output[:, 0, :]  # CLS token output


In [ ]:
class UnidirectionalAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads, direction="left-to-right"):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)
        self.direction = direction

    def forward(self, embeddings):
        seq_len = embeddings.size(1)
        mask = torch.tril(torch.ones(seq_len, seq_len)) if self.direction == "left-to-right" else torch.triu(torch.ones(seq_len, seq_len))
        mask = mask.to(embeddings.device).unsqueeze(0)
        attn_output, _ = self.attention(embeddings, embeddings, embeddings, attn_mask=mask)
        return attn_output[:, 0, :]  # CLS token output


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load pre-trained BERT
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to("cuda")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Zero-shot evaluation
def evaluate_bert(batch):
    inputs = bert_tokenizer(batch["sentence"], return_tensors="pt", padding=True, truncation=True, max_length=128).to("cuda")
    outputs = bert_model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    return predictions


---------
### References

[1] Raffel, Colin, Noam Shazeer, Adam Roberts, et al. (2020). *Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer*. [Link to paper](https://arxiv.org/abs/1910.10683)

[2] Touvron, Hugo, et al. (2023). *LLaMA 2: Open Foundation and Fine-Tuned Chat Models*. [Link to paper](https://arxiv.org/abs/2307.09288)

<span style="color:yellow;">*For further reading on this field of research, you can refer to the following papers:*</span>

[3] BehnamGhader, Adlakha, et al. (2024). *LLM2Vec: Large Language Models Are Secretly Powerful Text Encoders*. [Link to paper](https://arxiv.org/abs/2404.05961)

[4] Gao, Tianyu, et al. (2021). *SimCSE: Simple Contrastive Learning of Sentence Embeddings*. [Link to paper](https://arxiv.org/abs/2104.08821)

[5] Lee, et al. (2023). *NV-Embed: Improved Techniques for Training LLMs as Generalist Embedding Models*. [Link to paper](https://arxiv.org/abs/2405.17428)




# **Best regards.**